# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [60]:
import numpy as np
import pandas as pd

transitions_df = pd.read_csv('twizzleflu-transitions.tsv', sep='\t')
rewards_df = pd.read_csv('twizzleflu-rewards.tsv', sep='\t')

In [61]:
# YOUR CHANGES HERE

states = sorted(transitions_df['state'].unique())
actions = sorted(transitions_df['action'].unique())
print(f"\nStates: {states}")
print(f"Actions: {actions}")

T = {}
for action in actions:
    T[action] = np.zeros((len(states), len(states)))
    action_data = transitions_df[transitions_df['action'] == action]
    for _, row in action_data.iterrows():
        s_idx = states.index(row['state'])
        s_next_idx = states.index(row['next_state'])
        T[action][s_idx, s_next_idx] = row['probability']

R = np.zeros((len(states), len(actions)))
for _, row in rewards_df.iterrows():
    s_idx = states.index(row['state'])
    a_idx = actions.index(row['action'])
    R[s_idx, a_idx] = row['reward']

print("\nReward matrix shape:", R.shape)


def value_iteration_fixed_policy(T_policy, R_policy, gamma=1.0, epsilon=1e-6, max_iter=10000):
    """
    Value iteration for a fixed policy (no discount factor).
    T_policy: transition matrix for the fixed policy (n_states x n_states)
    R_policy: reward vector for the fixed policy (n_states,)
    """
    n_states = len(R_policy)
    V = np.zeros(n_states)
    
    for iteration in range(max_iter):
        V_new = R_policy + gamma * T_policy @ V
        
        if np.max(np.abs(V_new - V)) < epsilon:
            print(f"Converged in {iteration + 1} iterations")
            break
        
        V = V_new
    
    return V


do_nothing_action = 'do-nothing'  
if do_nothing_action not in actions:
    print(f"\nWarning: '{do_nothing_action}' not found. Available actions: {actions}")
    print("Using first action as 'do nothing'")
    do_nothing_action = actions[0]

do_nothing_idx = actions.index(do_nothing_action)
T_nothing = T[do_nothing_action]
R_nothing = R[:, do_nothing_idx]

V_nothing = value_iteration_fixed_policy(T_nothing, R_nothing)

discomfort_nothing = -V_nothing


States: ['exposed-1', 'exposed-2', 'exposed-3', 'recovered', 'symptoms-1', 'symptoms-2', 'symptoms-3']
Actions: ['do-nothing', 'drink-oj', 'sleep-8']

Reward matrix shape: (7, 3)
Converged in 55 iterations


Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [62]:
# YOUR CHANGES HERE

do_nothing_df = pd.DataFrame({
    'state': states,
    'expected_discomfort': discomfort_nothing
})
do_nothing_df.to_csv('do-nothing-discomfort.tsv', sep='\t', index=False)
print("\n=== PART 1: Do Nothing Discomfort ===")
print(do_nothing_df)


=== PART 1: Do Nothing Discomfort ===
        state  expected_discomfort
0   exposed-1             3.413330
1   exposed-2             4.266664
2   exposed-3             5.333331
3   recovered            -0.000000
4  symptoms-1             6.666664
5  symptoms-2             5.000000
6  symptoms-3             1.666667


Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [63]:
# YOUR CHANGES HERE

def value_iteration_mdp(T, R, states, actions, gamma=1.0, epsilon=1e-6, max_iter=10000):
    """
    Value iteration for MDP to find optimal policy.
    Returns: optimal values and optimal policy
    """
    n_states = len(states)
    n_actions = len(actions)
    V = np.zeros(n_states)
    
    for iteration in range(max_iter):
        V_new = np.zeros(n_states)
        
        for s in range(n_states):
            Q_values = np.zeros(n_actions)
            for a in range(n_actions):
                action_name = actions[a]
                Q_values[a] = R[s, a] + gamma * T[action_name][s, :] @ V
            
            V_new[s] = np.max(Q_values)
        
        if np.max(np.abs(V_new - V)) < epsilon:
            print(f"Converged in {iteration + 1} iterations")
            break
        
        V = V_new
    
    policy = []
    for s in range(n_states):
        Q_values = np.zeros(n_actions)
        for a in range(n_actions):
            action_name = actions[a]
            Q_values[a] = R[s, a] + gamma * T[action_name][s, :] @ V
        
        optimal_action_idx = np.argmax(Q_values)
        policy.append(actions[optimal_action_idx])
    
    return V, policy

V_optimal, policy_optimal = value_iteration_mdp(T, R, states, actions)
discomfort_optimal = -V_optimal



Converged in 66 iterations


Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [64]:
# YOUR CHANGES HERE

optimal_actions_df = pd.DataFrame({
    'state': states,
    'action': policy_optimal
})
optimal_actions_df.to_csv('minimum-discomfort-actions.tsv', sep='\t', index=False)

Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [65]:
# YOUR CHANGES HERE

optimal_values_df = pd.DataFrame({
    'state': states,
    'expected_discomfort': discomfort_optimal
})

Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [66]:
# YOUR CHANGES HERE

optimal_values_df.to_csv('minimum-discomfort-values.tsv', sep='\t', index=False)
print(optimal_values_df)

        state  expected_discomfort
0   exposed-1             0.749999
1   exposed-2             1.499999
2   exposed-3             2.999998
3   recovered            -0.000000
4  symptoms-1             5.999997
5  symptoms-2             4.500000
6  symptoms-3             1.500000


Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick and 0 if the current state corresponds to being better.
To be clear, the action does not matter for this reward function.


In [67]:
# YOUR CHANGES HERE

better_states = [s for s in states if 'better' in s.lower() or 'healthy' in s.lower() or 'well' in s.lower()]
if not better_states:
    print("\nWarning: No 'better' state found. Assuming last state is 'better'")
    better_states = [states[-1]]

R_duration = np.zeros((len(states), len(actions)))
for s_idx, state in enumerate(states):
    for a_idx in range(len(actions)):
        if state in better_states:
            R_duration[s_idx, a_idx] = 0
        else:
            R_duration[s_idx, a_idx] = -1

Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [68]:
# YOUR CHANGES HERE

duration_rewards_list = []
for s_idx, state in enumerate(states):
    for a_idx, action in enumerate(actions):
        duration_rewards_list.append({
            'state': state,
            'action': action,
            'reward': R_duration[s_idx, a_idx]
        })

duration_rewards_df = pd.DataFrame(duration_rewards_list)
duration_rewards_df.to_csv('duration-rewards.tsv', sep='\t', index=False)


Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [69]:
# YOUR CHANGES HERE

V_duration, policy_duration = value_iteration_mdp(T, R_duration, states, actions)


Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [70]:
# YOUR CHANGES HERE

duration_actions_df = pd.DataFrame({
    'state': states,
    'action': policy_duration
})
duration_actions_df.to_csv('minimum-duration-actions.tsv', sep='\t', index=False)


Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [71]:
# YOUR CHANGES HERE

expected_sick_days = -V_duration


Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [72]:
# YOUR CHANGES HERE

duration_days_df = pd.DataFrame({
    'state': states,
    'expected_sick_days': expected_sick_days
})
duration_days_df.to_csv('minimum-duration-days.tsv', sep='\t', index=False)



Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [73]:
# YOUR CHANGES HERE

T_duration_policy = np.zeros((len(states), len(states)))
R_duration_policy = np.zeros(len(states))

for s_idx, state in enumerate(states):
    action = policy_duration[s_idx]
    T_duration_policy[s_idx, :] = T[action][s_idx, :]
    a_idx = actions.index(action)
    R_duration_policy[s_idx] = R[s_idx, a_idx]

V_duration_discomfort = value_iteration_fixed_policy(T_duration_policy, R_duration_policy)
discomfort_speed = -V_duration_discomfort



Converged in 73 iterations


Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [74]:
# YOUR CHANGES HERE

comparison_df = pd.DataFrame({
    'state': states,
    'speed_discomfort': discomfort_speed,
    'minimize_discomfort': discomfort_optimal
})
comparison_df.to_csv('policy-comparison.tsv', sep='\t', index=False)


Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.